In [19]:
import numpy as np

# funkcija koja pretvara koordinate u homogene. Ukoliko su vec homogene, normalizuje ih
def to_hom(p):
    p = np.asarray(p, dtype=float)
    if p.shape == (2,):
        return np.array([p[0], p[1], 1.0], dtype=float)
    elif p.shape == (3,):
        if p[2] == 0:
            return p
        return p / p[2]
    else:
        raise ValueError("Tačka mora biti dimenzije 2 ili 3.")

# prava kroz dve tacke
def line_through(p, q):
    p = to_hom(p)
    q = to_hom(q)
    return np.cross(p, q)

# presek pravih
def intersect_lines(l1, l2):
    p = np.cross(l1, l2)
    if abs(p[2]) < 1e-12:
        return p
    return p / p[2]


In [20]:
#funkcija za trazenje tezista trougla. Koristimo je za preciznije trazenje tacaka nedogleda.
def vanishing_from_lines(lines):
    pts = []
    for i in range(len(lines)):
        for j in range(i+1, len(lines)):
            p = intersect_lines(lines[i], lines[j])
            if abs(p[2]) > 1e-12:
                p = p / p[2]
            pts.append(p)

    pts = np.array(pts)
    cx = pts[:,0].mean()
    cy = pts[:,1].mean()
    return np.array([cx, cy, 1.0])

In [21]:
def osmoteme(points):
    if len(points) != 7:
        raise ValueError("Očekujem listu [p1,p2,p3,p5,p6,p7,p8].")
    
    p1, p2, p3, p5, p6, p7, p8 = [to_hom(p) for p in points]

    #smer 1-2
    l12 = line_through(p1, p2)
    l56 = line_through(p5, p6)
    l87 = line_through(p8, p7)
    vX  = vanishing_from_lines([l12, l56, l87])

    # smer 2-3
    l23 = line_through(p2, p3)
    l67 = line_through(p6, p7)
    l58 = line_through(p5, p8)
    vY  = vanishing_from_lines([l23, l67, l58])

    # smer 1-5 - nije nam potreban za nalazenje p4, ali demonstrira trazenje tacke nedogleda
    l15 = line_through(p1, p5)
    l26 = line_through(p2, p6)
    l37 = line_through(p3, p7)
    vZ  = vanishing_from_lines([l15, l26, l37])

    # konacno trazenje tacke p4
    l3 = line_through(p3, vX)
    l1 = line_through(p1, vY)
    p4 = intersect_lines(l3, l1)
    return to_hom(p4)

In [22]:
# tacke empirijski izabrane u alatu Paint
picked_points = [(74,507), (369, 668), (714,328), (67, 406), (370, 557), (727, 252), (483, 171)]

if len(picked_points) == 7:
    print("7 tačaka je izabrano, računam p4...")
    import numpy as np
    p4 = osmoteme(picked_points)
    print(picked_points)
    print("p4 =", p4)

# tacka koja se dobija je bliska (488, 247), sto deluje dovoljno verodostojno.

7 tačaka je izabrano, računam p4...
[(74, 507), (369, 668), (714, 328), (67, 406), (370, 557), (727, 252), (483, 171)]
p4 = [488.55576869 247.57577389   1.        ]
